# Demo MAGxHR_1B (magnetic field 50Hz)

> Authors: Ashley Smith
>
> Abstract: Access to the high rate (50Hz) magnetic data (level 1b product).

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import SwarmRequest
import datetime as dt
import matplotlib.pyplot as plt

In [ ]:
request = SwarmRequest()

## MAGX_HR_1B product information

The 50Hz measurements of the magnetic field vector (`B_NEC`) and total intensity (`F`).

Documentation:
- https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook/level-1b-product-definitions#MAGX_HR_1B_Product

Measurements are available through VirES as part of collections with names containing `MAGx_HR`, for each Swarm spacecraft:

In [ ]:
request.available_collections("MAG_HR", details=False)

The measurements can be used together with geomagnetic model evaluations as shall be shown below.

### Check what "MAG" data variables are available

In [ ]:
request.available_measurements("MAG_HR")

## Fetch data

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MAGA_HR_1B")
request.set_products(
   measurements=request.PRODUCT_VARIABLES['MAG_HR'],
   sampling_step="PT0.019S", # ~50Hz sampling
)
data = request.get_between(
   start_time="2015-06-21T12:00:00Z", 
   end_time="2015-06-21T12:01:00Z",
   asynchronous=False
)

In [ ]:
data.sources

In [ ]:
ds = data.as_xarray()
ds

In [ ]:
fig, axes = plt.subplots(figsize=(20, 3), ncols=3, sharex=True)
for i in range(3):
    axes[i].plot(ds["Timestamp"], ds["B_NEC"][:, i])
    axes[i].set_ylabel("NEC"[i])
    axes[i].set_xlabel("Time")
fig.suptitle("B_NEC components [nT]");

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(20, 3))
dBdt = np.diff(ds["B_NEC"], axis=0) * (1/50)
ax.plot(ds["Timestamp"][1:], dBdt[:, 2])
ax.set_ylabel("nT / s")
ax.set_xlabel("Time")
ax.set_title("Vertical component (C) first differences");